In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

In [2]:
df = pd.read_csv('~/code/patrickevans29/raw_data/complete_cleaned_dataset.csv')


In [3]:
df.head(2)

,ID,innings,overs,ballnumber,batter,bowler,non_striker,extra_type,batsman_run,extras_run,...,TossDecision,SuperOver,WinningTeam,WonBy,Margin,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,1,1,1,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,...,bat,n,gujarat titans,wickets,7.0,hh pandya,"['ybk jaiswal', 'jc buttler', 'sv samson', 'd ...","['wp saha', 'shubman gill', 'ms wade', 'hh pan...",cb gaffaney,nitin menon
1,1312200,1,1,2,ybk jaiswal,mohammed shami,jc buttler,legbyes,0,1,...,bat,n,gujarat titans,wickets,7.0,hh pandya,"['ybk jaiswal', 'jc buttler', 'sv samson', 'd ...","['wp saha', 'shubman gill', 'ms wade', 'hh pan...",cb gaffaney,nitin menon


In [4]:
df.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non_striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'City', 'Date', 'Season',
       'MatchNumber', 'Team1', 'Team2', 'Venue', 'TossWinner', 'TossDecision',
       'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'Player_of_Match',
       'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2'],
      dtype='object')

In [5]:
df['Team1Players'] = df['Team1Players'].apply(lambda x: ast.literal_eval(x))
df['Team2Players'] = df['Team2Players'].apply(lambda x: ast.literal_eval(x))

In [6]:
def player_batting_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the ball by ball match cleaned dataframe as a base.
    Creates a dataframe of player batting features extracted from the
    ball by ball match dataframe.
    Returns the features processed.
    """
    assert isinstance(df, pd.DataFrame)

    df = df[PLAYER_PROCESSING_COLUMNS]

    ### Total Runs
    batter_total_runs = df.groupby(['batter'], as_index=False)['batsman_run']\
        .sum('batsman_run').rename(columns={'batsman_run': 'total_runs'})

    ### Balls Faced
    balls_faced = df[df['extra_type'].isin(['noextra', 'legbyes', 'byes', 'noballs'])]\
                .groupby(['batter'], as_index=False)['ballnumber'].count()\
                .rename(columns={'ballnumber': 'balls_faced'})

    ### Batter Innings
    batter_innings = df.groupby(['batter'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'batter_innings'})
    non_striker_innings = df.groupby(['non_striker'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'non_striker_innings', 'non_striker': 'batter'})
    combined_innings_df = batter_innings.merge(non_striker_innings, \
                    on='batter', how='outer').fillna(0)
    combined_innings_df['bat_innings'] = combined_innings_df[['batter_innings', 'non_striker_innings']].max(axis=1)
    bat_innings = combined_innings_df.drop(columns=['batter_innings', 'non_striker_innings'])

    ### Boundaries
    if len(df[df['non_boundary'] == 1]) > 0:
        non_boundary = df[df['non_boundary'] == 1].index.tolist()
        temp_df = df.drop(index=non_boundary)
    else:
        temp_df = df.copy()
    fours = temp_df[temp_df['batsman_run'] == 4].groupby(['batter'], as_index=False)\
            ['batsman_run'].count().rename(columns={'batsman_run': 'fours'})
    sixes = temp_df[temp_df['batsman_run'] == 6].groupby(['batter'], as_index=False)\
            ['batsman_run'].count().rename(columns={'batsman_run': 'sixes'})

    ### Batsman Totals
    batsman_score_game = df.groupby(['ID', 'batter'], as_index=False)\
                        ['batsman_run'].sum().drop(columns='ID')
    fifty = batsman_score_game[batsman_score_game['batsman_run'] >= 50]\
        .groupby(['batter'], as_index=False).count().rename(columns={'batsman_run': '50s'})
    hundred = batsman_score_game[batsman_score_game['batsman_run'] >= 100]\
        .groupby(['batter'], as_index=False).count().rename(columns={'batsman_run': '100s'})
    merged_totals = fifty.merge(hundred, on='batter', how='outer').fillna(0)
    merged_totals['50s'] = merged_totals['50s'].sub(merged_totals['100s']).astype('int')
    merged_totals['100s'] = merged_totals['100s'].astype('int')

    ### Batsman Not Out
    last_ball_index_temp = df.groupby(['ID', 'innings'], as_index=False)['innings']\
                        .idxmax().sort_values(by='innings', ascending=False)['innings'].tolist()
    last_ball_index_temp = last_ball_index_temp[:-1]
    last_ball_index = [index - 1 for index in last_ball_index_temp]
    last_ball_index.insert(0, df.shape[0])
    player_out_final_ball = df[(df['isWicketDelivery']==1) & (df.index.isin(last_ball_index))]\
                            .groupby(['player_out'], as_index=False)['ID'].count()\
                            .rename(columns={'ID': 'out_count', 'player_out': 'batter'})
    final_ball_batter = df[df.index.isin(last_ball_index)].groupby(['batter'], as_index=False)\
                        ['ID'].count().rename(columns={'ID': 'at_bat_count'})
    final_ball_non_striker = df[df.index.isin(last_ball_index)].groupby(['non_striker'], as_index=False)\
                            ['ID'].count().rename(columns={'ID': 'non_striker_count', 'non_striker': 'batter'})
    combined_final_ball = final_ball_batter.merge(final_ball_non_striker, on='batter', how='outer')\
                            .merge(player_out_final_ball, on='batter', how='outer')
    combined_final_ball.fillna(0, inplace=True)
    combined_final_ball['not_out'] = combined_final_ball['at_bat_count']\
                                    .add(combined_final_ball['non_striker_count'])\
                                    .sub(combined_final_ball['out_count']).astype('int')
    not_out = combined_final_ball[['batter', 'not_out']]

    ### High Score
    high_score = df.groupby(['ID', 'batter'], as_index=False)['batsman_run']\
                        .sum().groupby('batter', as_index=False)['batsman_run'].max()\
                        .rename(columns={'batsman_run': 'high_score'})

    ### Merge
    batting_stats_merged = bat_innings.merge(batter_total_runs, on='batter', how='outer')\
                        .merge(not_out, on='batter', how='outer')\
                        .merge(merged_totals, on='batter', how='outer')\
                        .merge(fours, on='batter', how='outer')\
                        .merge(sixes, on='batter', how='outer')\
                        .merge(high_score, on='batter', how='outer')\
                        .merge(balls_faced, on='batter', how='outer').fillna(0)

    ### Feature creation
    batting_stats_merged['average_score'] = round\
        ((batting_stats_merged['total_runs'] / batting_stats_merged['bat_innings']), 2).fillna(0)
    batting_stats_merged['batting_average'] = round((batting_stats_merged['total_runs'] / \
        (batting_stats_merged['bat_innings'] - batting_stats_merged['not_out'])), 2).fillna(0)
    batting_stats_merged['batting_average'] = np.where(batting_stats_merged\
        ['batting_average'] == np.inf, batting_stats_merged['average_score'], batting_stats_merged['batting_average'])
    batting_stats_merged['batting_strike_rate'] = round\
        ((batting_stats_merged['total_runs'] * 100)/batting_stats_merged['balls_faced'], 2).fillna(0)

    batting_stats_merged = batting_stats_merged.rename(columns={'batter': 'player'})

    print("✅ Batting Features Created")

    return batting_stats_merged


def player_bowling_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the ball by ball match cleaned dataframe as a base.
    Creates a dataframe of player bowling features extracted from the
    ball by ball match dataframe.
    Returns the features processed.
    """
    assert isinstance(df, pd.DataFrame)

    df = df[PLAYER_PROCESSING_COLUMNS]

    ### Bowler Innings
    bowler_innings = df.groupby(['bowler'], as_index=False)['ID'].nunique()\
                    .rename(columns={'ID': 'bowler_innings'})

    ### Balls Bowled
    balls_bowled = df[df['extra_type'].isin(['noextra', 'legbyes', 'byes', 'penalty'])]\
                .groupby(['bowler'], as_index=False)['ballnumber'].count()\
                .rename(columns={'ballnumber': 'balls_bowled'})

    ### Runs Conceded
    runs_conceded = df[df['extra_type'].isin(['noextra', 'wides', 'noballs'])]\
                .groupby(['bowler'], as_index=False)['total_run'].sum()\
                .rename(columns={'total_run': 'runs_conceded'})

    ### Wickets Taken
    wickets = df[(df['isWicketDelivery'] == 1) & (df['kind'].isin\
                (['nokind', 'caught', 'caught and bowled', 'bowled','stumped', 'lbw', 'hit wicket']))]\
                .groupby(['bowler'], as_index=False)['isWicketDelivery'].count()\
                .rename(columns={'isWicketDelivery': 'wickets'})

    ### Wickets Per Game Stats
    wicket_count_per_game = df[(df['isWicketDelivery'] == 1) & (df['kind'].isin\
                (['nokind', 'caught', 'caught and bowled', 'bowled','stumped', 'lbw', 'hit wicket']))]\
                .groupby(['ID', 'bowler'], as_index=False)['isWicketDelivery'].count()\
                .rename(columns={'isWicketDelivery': 'wickets'})

    four_wickets = wicket_count_per_game[wicket_count_per_game['wickets'] == 4]\
                .groupby(['bowler'], as_index=False)['wickets'].count()\
                .rename(columns={'wickets': 'four_wickets'})

    five_wickets = wicket_count_per_game[wicket_count_per_game['wickets'] == 5]\
                .groupby(['bowler'], as_index=False)['wickets'].count()\
                .rename(columns={'wickets': 'five_wickets'})

    ### Merge
    bowling_stats_merged = bowler_innings.merge(balls_bowled, on='bowler', how='outer')\
                        .merge(runs_conceded, on='bowler', how='outer')\
                        .merge(wickets, on='bowler', how='outer')\
                        .merge(four_wickets, on='bowler', how='outer')\
                        .merge(five_wickets, on='bowler', how='outer').fillna(0)

    ### Feature Creation
    bowling_stats_merged['bowling_average'] = round\
        ((bowling_stats_merged['runs_conceded'] / bowling_stats_merged['wickets']), 2).fillna(0)
    bowling_stats_merged['bowling_average'] = np.where\
        (bowling_stats_merged['bowling_average'] == np.inf, 0, bowling_stats_merged['bowling_average'])
    bowling_stats_merged['bowling_economy_rate'] = round\
        ((bowling_stats_merged['runs_conceded']/(bowling_stats_merged['balls_bowled'] / 6)), 2)
    bowling_stats_merged['bowling_strike_rate'] = round((bowling_stats_merged['balls_bowled']/bowling_stats_merged['wickets']), 2)
    bowling_stats_merged['bowling_strike_rate'] = np.where\
        (bowling_stats_merged['bowling_strike_rate'] == np.inf, 0, bowling_stats_merged['bowling_strike_rate'])

    bowling_stats_merged = bowling_stats_merged.rename(columns={'bowler': 'player'})

    print("✅ Bowling Features Created")

    return bowling_stats_merged


def win_loss_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a cleaned dataframe as a base.
    Creates a dataframe of player win loss statistics.
    Returns the features processed.
    """
    assert isinstance(df, pd.DataFrame)

    df = df[WIN_LOSS_PROCESSING_COLUMNS]

    ### Group to remove duplicates
    df = df.groupby('ID', as_index=False).first()

    ### Team 1 Wins
    def team_1_wins(Team1, Team2, WinningTeam):
        if WinningTeam == Team1:
            return 1
        elif WinningTeam == Team2:
            return 0
        else:
            return -1

    df['team_1_win'] = df.apply(lambda row: \
        team_1_wins(row['Team1'], row['Team2'], row['WinningTeam']), axis=1)

    ### Winning Player Count
    def winning_team_players(team_1_win, Team1Players, Team2Players):
        if team_1_win == 1:
            return Team1Players
        elif team_1_win == 0:
            return Team2Players
        else:
            return []

    df['winning_team_players'] = df.apply(lambda row: winning_team_players\
        (row['team_1_win'], row['Team1Players'], row['Team2Players']), axis=1)

    player_wins = []
    def player_wins_list(winning_team_players):
        for row in winning_team_players:
            for player in row:
                player_wins.append(player)
        return player_wins

    player_wins_list(df['winning_team_players'])

    player_win_count = {}
    for player in player_wins:
        if player in player_win_count:
            player_win_count[player] += 1
        else:
            player_win_count[player] = 1

    wins = pd.DataFrame({'player': player_win_count.keys(), \
                        'wins': player_win_count.values()})

    ### Losing Player Count
    def losing_team_players(team_1_win, Team1Players, Team2Players):
        if team_1_win == 0:
            return Team1Players
        elif team_1_win == 1:
            return Team2Players
        else:
            return []

    df['losing_team_players'] = df.apply(lambda row: losing_team_players\
        (row['team_1_win'], row['Team1Players'], row['Team2Players']), axis=1)

    player_loses = []
    def player_loses_list(losing_team_players):
        for row in losing_team_players:
            for player in row:
                player_loses.append(player)
        return player_loses

    player_loses_list(df['losing_team_players'])

    player_loss_count = {}
    for player in player_loses:
        if player in player_loss_count:
            player_loss_count[player] += 1
        else:
            player_loss_count[player] = 1

    loses = pd.DataFrame({'player': player_loss_count.keys(), \
                            'loses': player_loss_count.values()})

    ### Matches
    matches = {}

    for player in player_win_count:
        if player in player_loss_count:
            matches[player] = player_win_count[player] + player_loss_count[player]
        elif player not in player_loss_count:
            matches[player] = player_win_count[player]

    for player in player_loss_count:
        if player not in player_win_count:
            matches[player] = player_loss_count[player]

    matches = pd.DataFrame({'player': matches.keys(), \
                            'matches': matches.values()})

    ### Merge
    win_loss_merge = matches.merge(wins, on='player', how='outer')\
                        .merge(loses, on='player', how='outer').fillna(0)

    ### Create Win Ratio
    win_loss_merge['win_ratio'] = \
        round((win_loss_merge['wins']/win_loss_merge['matches']), 2)

    print("✅ Win Loss Features Created")

    return win_loss_merge


def player_features_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a cleaned dataframe as a base.
    Compiles the final player dataset.
    """
    assert isinstance(df, pd.DataFrame)

    batting = player_batting_features(df)
    bowling = player_bowling_features(df)
    win_loss = win_loss_features(df)

    final_player_dataset = batting.merge(bowling, on='player', how='outer')\
                            .merge(win_loss, on='player', how='outer').fillna(0)

    print("✅ Final Player Dataset Created")

    return final_player_dataset

In [8]:
PLAYER_PROCESSING_COLUMNS = ['ID', 'innings', 'overs', 'ballnumber', 'batter',
        'bowler', 'non_striker', 'extra_type', 'batsman_run', 'extras_run',
        'total_run', 'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
        'fielders_involved', 'BattingTeam']

WIN_LOSS_PROCESSING_COLUMNS = ['ID', 'Season', 'Team1', 'Team2', 'WinningTeam',
                             'Player_of_Match', 'Team1Players', 'Team2Players']

In [343]:
team_1 = []
team_2 = []

for match in chunk_id:
    chunk = df[df['ID']==match]
    team_1_chunk = chunk['Team1Players'].iloc[0]
    team_2_chunk = chunk['Team2Players'].iloc[0]
    player_chunk = player_features_dataset(chunk)
    t1 = player_chunk[player_chunk['player'].isin(team_1_chunk)]
    t2 = player_chunk[player_chunk['player'].isin(team_2_chunk)]

    t1.loc[:, 'ID'] = match
    t2.loc[:, 'ID'] = match
        
    team_1.append(t1)
    team_2.append(t2)

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created


/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/123114604.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/123114604.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/123114604.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [345]:
pd.concat(team_1, ignore_index=True)

,player,bat_innings,total_runs,not_out,50s,100s,fours,sixes,high_score,balls_faced,...,four_wickets,five_wickets,bowling_average,bowling_economy_rate,bowling_strike_rate,matches,wins,loses,win_ratio,ID
0,d padikkal,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,10.0,...,0.0,0.0,0.00,0.00,0.0,1,0.0,1.0,0.0,1312200
1,jc buttler,1.0,39.0,0.0,0.0,0.0,5.0,0.0,39.0,35.0,...,0.0,0.0,0.00,0.00,0.0,1,0.0,1.0,0.0,1312200
2,oc mccoy,1.0,8.0,0.0,0.0,0.0,0.0,1.0,8.0,5.0,...,0.0,0.0,0.00,8.21,0.0,1,0.0,1.0,0.0,1312200
3,r ashwin,1.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,9.0,...,0.0,0.0,0.00,10.67,0.0,1,0.0,1.0,0.0,1312200
4,r parag,1.0,15.0,0.0,0.0,0.0,1.0,0.0,15.0,15.0,...,0.0,0.0,0.00,0.00,0.0,1,0.0,1.0,0.0,1312200
5,so hetmyer,1.0,11.0,0.0,0.0,0.0,2.0,0.0,11.0,12.0,...,0.0,0.0,0.00,0.00,0.0,1,0.0,1.0,0.0,1312200
6,sv samson,1.0,14.0,0.0,0.0,0.0,2.0,0.0,14.0,11.0,...,0.0,0.0,0.00,0.00,0.0,1,0.0,1.0,0.0,1312200
7,ta boult,1.0,11.0,0.0,0.0,0.0,0.0,1.0,11.0,7.0,...,0.0,0.0,14.00,3.50,24.0,1,0.0,1.0,0.0,1312200
8,ybk jaiswal,1.0,22.0,0.0,0.0,0.0,1.0,2.0,22.0,16.0,...,0.0,0.0,0.00,0.00,0.0,1,0.0,1.0,0.0,1312200
9,m prasidh krishna,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,40.00,10.00,24.0,1,0.0,1.0,0.0,1312200


In [ ]:
for player in team_1_chunk:
    chunk_reduced[]

In [277]:
player_chunk.head(2)

,player,bat_innings,total_runs,not_out,50s,100s,fours,sixes,high_score,balls_faced,...,wickets,four_wickets,five_wickets,bowling_average,bowling_economy_rate,bowling_strike_rate,matches,wins,loses,win_ratio
0,dj hooda,1.0,45.0,0.0,0.0,0.0,1.0,4.0,45.0,26.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,1.0,0.0
1,e lewis,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,1.0,0.0


In [278]:
chunk.head(2)

,ID,innings,overs,ballnumber,batter,bowler,non_striker,extra_type,batsman_run,extras_run,...,TossDecision,SuperOver,WinningTeam,WonBy,Margin,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
469,1312198,1,1,1,v kohli,mohsin khan,f du plessis,noextra,0,0,...,field,n,royal challengers bangalore,runs,14.0,rm patidar,"[v kohli, f du plessis, rm patidar, gj maxwell...","[q de kock, kl rahul, m vohra, dj hooda, mp st...",j madanagopal,ma gough
470,1312198,1,1,2,v kohli,mohsin khan,f du plessis,noextra,0,0,...,field,n,royal challengers bangalore,runs,14.0,rm patidar,"[v kohli, f du plessis, rm patidar, gj maxwell...","[q de kock, kl rahul, m vohra, dj hooda, mp st...",j madanagopal,ma gough


In [347]:
t1

,player,bat_innings,total_runs,not_out,50s,100s,fours,sixes,high_score,balls_faced,...,four_wickets,five_wickets,bowling_average,bowling_economy_rate,bowling_strike_rate,matches,wins,loses,win_ratio,ID
2,f du plessis,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.00,0.00,0.0,1,1.0,0.0,1.0,1312198
3,gj maxwell,1.0,9.0,0.0,0.0,0.0,0.0,1.0,9.0,10.0,...,0.0,0.0,0.00,0.00,0.0,1,1.0,0.0,1.0,1312198
4,kd karthik,1.0,37.0,1.0,0.0,0.0,5.0,1.0,37.0,23.0,...,0.0,0.0,0.00,0.00,0.0,1,1.0,0.0,1.0,1312198
8,mk lomror,1.0,14.0,0.0,0.0,0.0,2.0,0.0,14.0,9.0,...,0.0,0.0,0.00,0.00,0.0,1,1.0,0.0,1.0,1312198
12,rm patidar,1.0,112.0,1.0,0.0,1.0,12.0,7.0,112.0,54.0,...,0.0,0.0,0.00,0.00,0.0,1,1.0,0.0,1.0,1312198
13,v kohli,1.0,25.0,0.0,0.0,0.0,2.0,0.0,25.0,24.0,...,0.0,0.0,0.00,0.00,0.0,1,1.0,0.0,1.0,1312198
15,hv patel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.00,6.25,24.0,1,1.0,0.0,1.0,1312198
16,jr hazlewood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,14.33,10.75,8.0,1,1.0,0.0,1.0,1312198
17,mohammed siraj,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,41.00,10.25,24.0,1,1.0,0.0,1.0,1312198
19,pwh de silva,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,42.00,10.50,24.0,1,1.0,0.0,1.0,1312198


In [339]:
test = t1.iloc[:, 1:].groupby('ID').mean().round(2)

In [340]:
test.columns

Index(['bat_innings', 'total_runs', 'not_out', '50s', '100s', 'fours', 'sixes',
       'high_score', 'balls_faced', 'average_score', 'batting_average',
       'batting_strike_rate', 'bowler_innings', 'balls_bowled',
       'runs_conceded', 'wickets', 'four_wickets', 'five_wickets',
       'bowling_average', 'bowling_economy_rate', 'bowling_strike_rate',
       'matches', 'wins', 'loses', 'win_ratio'],
      dtype='object')

In [341]:
test.head(2)

,bat_innings,total_runs,not_out,50s,100s,fours,sixes,high_score,balls_faced,average_score,...,wickets,four_wickets,five_wickets,bowling_average,bowling_economy_rate,bowling_strike_rate,matches,wins,loses,win_ratio
ID,,,,,,,,,,,,,,,,,,,,,
1312198,0.55,17.91,0.18,0.0,0.09,1.91,0.82,17.91,11.0,17.91,...,0.55,0.0,0.0,11.12,4.23,7.27,1.0,1.0,0.0,1.0


In [355]:
def team_average(df, t1, t2):
    columns = t1.drop(columns='player').columns
    for column in columns:
        df[f"Team_1_{column}"] = t1[column].mean()
        df[f"Team_2_{column}"] = t2[column].mean()
    return df

In [356]:
df2 = team_average(df, t1, t2)

In [360]:
df2[['Team_1_batting_average', 'Team_2_batting_average',
       'Team_1_batting_strike_rate', 'Team_2_batting_strike_rate',
       'Team_1_bowler_innings', 'Team_2_bowler_innings', 'Team_1_balls_bowled',
       'Team_2_balls_bowled', 'Team_1_runs_conceded', 'Team_2_runs_conceded',
       'Team_1_wickets', 'Team_2_wickets']]

,Team_1_batting_average,Team_2_batting_average,Team_1_batting_strike_rate,Team_2_batting_strike_rate,Team_1_bowler_innings,Team_2_bowler_innings,Team_1_balls_bowled,Team_2_balls_bowled,Team_1_runs_conceded,Team_2_runs_conceded,Team_1_wickets,Team_2_wickets
0,17.909091,15.545455,65.273636,91.85,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636
1,17.909091,15.545455,65.273636,91.85,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636
2,17.909091,15.545455,65.273636,91.85,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636
3,17.909091,15.545455,65.273636,91.85,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636
4,17.909091,15.545455,65.273636,91.85,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636
...,...,...,...,...,...,...,...,...,...,...,...,...
225949,17.909091,15.545455,65.273636,91.85,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636
225950,17.909091,15.545455,65.273636,91.85,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636
225951,17.909091,15.545455,65.273636,91.85,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636
225952,17.909091,15.545455,65.273636,91.85,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636


In [361]:
number_of_matches = 5

In [364]:
df[df['ID'].isin(range(number_of_matches))]

,ID,innings,overs,ballnumber,batter,bowler,non_striker,extra_type,batsman_run,extras_run,...,Team_1_matches,Team_2_matches,Team_1_wins,Team_2_wins,Team_1_loses,Team_2_loses,Team_1_win_ratio,Team_2_win_ratio,Team_1_ID,Team_2_ID


In [366]:
type(match_id[0:number_of_matches])

list

In [460]:
def match_filter(df: pd.DataFrame,
               number_of_matches: int = "all",
               team_1: list = "all",
               team_2: list = "all",
               season: list = "all") -> pd.DataFrame:
    """
    Takes a cleaned dataframe as a base.
    Filters the dataframe by number of targets:
    
    number_of_matches, the maximum number of matches
    team_1, the list of teams to include as team_1
    team_2, the list of teams to include as team_2
    season, the list of seasons to include in the filter.
    
    Returns a filtered dataframe
    """

    if team_1 != "all":
        df = df[df['Team1'].isin(team_1)]

    if team_2 != "all":
        df = df[df['Team2'].isin(team_2)]

    if season != "all":
        df = df[df['Season'].isin(season)]

    if number_of_matches != "all":
        match_id = df['ID'].unique().tolist()
        df = df[df['ID'].isin(match_id[0:number_of_matches])]

    return df


def player_team_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a cleaned dataframe as a base.
    
    Processes the dataframe to output player data using the feature engineer
    module
    
    Returns a dataframe for both team_1 and team_2 players
    """

    match_id = df['ID'].unique().tolist()

    team_1_list = []
    team_2_list = []

    for match in match_id:
        match_df = df[df['ID']==match]

        team_1_players = match_df['Team1Players'].iloc[0]
        team_2_players = match_df['Team2Players'].iloc[0]

        player_match_df = player_features_dataset(match_df)

        t1 = player_match_df[player_match_df['player'].isin(team_1_players)]
        t2 = player_match_df[player_match_df['player'].isin(team_2_players)]

        t1.loc[:, 'ID'] = match
        t2.loc[:, 'ID'] = match

        team_1_list.append(t1)
        team_2_list.append(t2)

    team_1 = pd.concat(team_1_list, ignore_index=True)
    team_2 = pd.concat(team_2_list, ignore_index=True)

    return team_1, team_2


def team_average(df_filtered: pd.DataFrame, 
                 team_1: pd.DataFrame, 
                 team_2: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the filtered dataframe from match_filter()
    and team_1, team_2 dataframe from player_team_data()
    
    Processes the dataframe to output a combined dataframe 
    with average scores for each team grouped by the match ID
    
    Returns a combined dataframe.
    """
    
    df = df_filtered[WIN_LOSS_PROCESSING_COLUMNS].groupby('ID').first()
    columns = team_1.drop(columns='player').columns
    
    for column in columns:
        df[f"Team_1_{column}"] = team_1.groupby('ID')[column].mean()
        df[f"Team_2_{column}"] = team_2.groupby('ID')[column].mean()
        
    return df


def game_average(df: pd.DataFrame,
                number_of_matches: int = "all",
                team_1: list = "all",
                team_2: list = "all",
                season: list = "all") -> pd.DataFrame:
    """
    Takes a cleaned dataframe as a base.
    
    A combined process for returning the average match data for a filtered
    dataframe using:
    match_filter()
    player_team_data()
    team_average()
    
    number_of_matches, the maximum number of matches
    team_1, the list of teams to include as team_1
    team_2, the list of teams to include as team_2
    season, the list of seasons to include in the filter.
    
    Returns a dataframe
    """

    df_filtered = match_filter(df, number_of_matches, team_1, team_2, season)

    team_1_df, team_2_df = player_team_data(df_filtered)

    average_df = team_average(df_filtered, team_1_df, team_2_df)

    return average_df


In [462]:
test = chunk_df(df)

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created


/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created


/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created


...

/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created


/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created


/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1811067975.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [465]:
test[['Team_1_average_score', 'Team_2_average_score',
       'Team_1_batting_average', 'Team_2_batting_average',
       'Team_1_batting_strike_rate', 'Team_2_batting_strike_rate',
       'Team_1_bowler_innings', 'Team_2_bowler_innings', 'Team_1_balls_bowled',
       'Team_2_balls_bowled', 'Team_1_runs_conceded', 'Team_2_runs_conceded',
       'Team_1_wickets', 'Team_2_wickets']]

,Team_1_average_score,Team_2_average_score,Team_1_batting_average,Team_2_batting_average,Team_1_batting_strike_rate,Team_2_batting_strike_rate,Team_1_bowler_innings,Team_2_bowler_innings,Team_1_balls_bowled,Team_2_balls_bowled,Team_1_runs_conceded,Team_2_runs_conceded,Team_1_wickets,Team_2_wickets
ID,,,,,,,,,,,,,,
335982,5.727273,18.636364,5.727273,18.636364,61.249091,60.585455,0.545455,0.454545,10.909091,8.272727,19.454545,6.727273,0.272727,0.818182
335983,17.818182,21.272727,17.818182,21.272727,78.687273,106.920000,0.636364,0.454545,10.909091,10.909091,21.636364,18.272727,0.454545,0.363636
335984,11.090909,11.090909,11.090909,11.090909,50.265455,86.576364,0.454545,0.545455,10.909091,8.272727,11.363636,12.000000,0.545455,0.090909
335985,14.000000,14.636364,14.000000,14.636364,86.033636,87.982727,0.545455,0.545455,10.727273,10.909091,15.090909,14.272727,0.454545,0.454545
335986,7.636364,9.090909,7.636364,9.090909,41.866364,60.210000,0.545455,0.545455,10.181818,10.363636,9.636364,9.090909,0.818182,0.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304116,13.181818,14.000000,13.181818,14.000000,80.097273,125.851818,0.545455,0.454545,8.272727,10.909091,14.363636,14.000000,0.454545,0.636364
1312197,16.090909,16.181818,16.090909,16.181818,82.515455,55.494545,0.454545,0.545455,10.636364,10.909091,17.272727,17.090909,0.181818,0.363636
1312198,17.909091,15.545455,17.909091,15.545455,65.273636,91.850000,0.454545,0.454545,10.909091,10.909091,16.909091,18.818182,0.545455,0.363636


In [451]:
df_filtered = match_filter(df, number_of_matches=10, team_1=['chennai super kings'])

In [428]:
team_1, team_2 = player_team_data(df_filtered)

✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created


/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1274588974.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1274588974.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1274588974.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created
✅ Batting Features Created
✅ Bowling Features Created
✅ Win Loss Features Created
✅ Final Player Dataset Created


/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1274588974.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1274588974.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.loc[:, 'ID'] = match
/var/folders/4n/3hw1gckd4zg3786ftn8gpp8c0000gn/T/ipykernel_23023/1274588974.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [458]:
average = team_average(filter, team_1, team_2)

In [459]:
average

,Season,Team1,Team2,WinningTeam,Player_of_Match,Team1Players,Team2Players,Team_1_bat_innings,Team_2_bat_innings,Team_1_total_runs,...,Team_1_matches,Team_2_matches,Team_1_wins,Team_2_wins,Team_1_loses,Team_2_loses,Team_1_win_ratio,Team_2_win_ratio,Team_1_ID,Team_2_ID
ID,,,,,,,,,,,,,,,,,,,,,
1304047,2022,chennai super kings,kolkata knight riders,kolkata knight riders,ut yadav,"[rd gaikwad, dp conway, rv uthappa, at rayudu,...","[am rahane, vr iyer, n rana, ss iyer, sw billi...",0.636364,0.545455,11.363636,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1304047.0,1304047.0
1304053,2022,chennai super kings,lucknow super giants,lucknow super giants,e lewis,"[rv uthappa, rd gaikwad, mm ali, s dube, at ra...","[kl rahul, q de kock, mk pandey, e lewis, dj h...",0.818182,0.545455,17.818182,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1304053.0,1304053.0
1304063,2022,chennai super kings,sunrisers hyderabad,sunrisers hyderabad,abhishek sharma,"[rv uthappa, rd gaikwad, mm ali, at rayudu, s ...","[abhishek sharma, ks williamson, ra tripathi, ...",0.818182,0.363636,13.545455,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1304063.0,1304063.0
1304068,2022,chennai super kings,royal challengers bangalore,chennai super kings,s dube,"[rd gaikwad, rv uthappa, mm ali, s dube, ra ja...","[f du plessis, anuj rawat, v kohli, gj maxwell...",0.545455,1.000000,18.454545,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1304068.0,1304068.0
1304075,2022,chennai super kings,gujarat titans,gujarat titans,da miller,"[rd gaikwad, rv uthappa, mm ali, at rayudu, s ...","[wp saha, shubman gill, v shankar, a manohar, ...",0.545455,0.818182,14.909091,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1304075.0,1304075.0
1304092,2022,chennai super kings,sunrisers hyderabad,chennai super kings,rd gaikwad,"[rd gaikwad, dp conway, ms dhoni, ra jadeja, r...","[abhishek sharma, ks williamson, ra tripathi, ...",0.363636,0.727273,17.545455,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1304092.0,1304092.0
1304101,2022,chennai super kings,delhi capitals,chennai super kings,dp conway,"[rd gaikwad, dp conway, s dube, at rayudu, ms ...","[da warner, ks bharat, mr marsh, rr pant, r po...",0.727273,1.000000,17.818182,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1304101.0,1304101.0
1304105,2022,chennai super kings,mumbai indians,mumbai indians,dr sams,"[rd gaikwad, dp conway, mm ali, rv uthappa, at...","[ishan kishan, rg sharma, dr sams, tilak varma...",1.000000,0.636364,7.454545,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1304105.0,1304105.0
1304108,2022,chennai super kings,gujarat titans,gujarat titans,wp saha,"[rd gaikwad, dp conway, mm ali, n jagadeesan, ...","[wp saha, shubman gill, ms wade, hh pandya, da...",0.636364,0.454545,11.454545,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1304108.0,1304108.0


In [457]:
filter = df_filtered[WIN_LOSS_PROCESSING_COLUMNS].groupby('ID').first()

In [446]:
columns = team_1.drop(columns='player').columns
for column in columns:
    df_filtered[f"Team_1_{column}"] = team_1.groupby('ID')[column].mean()

In [453]:
df_filtered.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non_striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'City', 'Date', 'Season',
       'MatchNumber', 'Team1', 'Team2', 'Venue', 'TossWinner', 'TossDecision',
       'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'Player_of_Match',
       'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2'],
      dtype='object')

In [449]:
df_filtered.isna().sum()

ID                     0
innings                0
overs                  0
ballnumber             0
batter                 0
                    ... 
Team_1_matches      2400
Team_1_wins         2400
Team_1_loses        2400
Team_1_win_ratio    2400
Team_1_ID           2400
Length: 61, dtype: int64